### Sharon Laurance Muthipeedika
### 312486

In [3]:
#Import required Libraries
from torch.utils.data import Dataset,DataLoader
import torch
import os
import torch.nn as nn
import cv2
from math import sqrt
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
import warnings
warnings.filterwarnings("ignore")

In [4]:
class Dataset(Dataset):
    
    def __init__(self):
        self.filepath='/kaggle/input/car-steering-angle-prediction/driving_dataset/'
        self.filenames= os.listdir(self.filepath)
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, index):
        
        filename=self.filenames[index]
        img= cv2.imread(self.filepath+filename)
        
        
        filename=self.filenames[0]
        img0= cv2.imread(self.filepath+filename)

        
        if img is None:
            print('Wrong path:',self.filepath) #If the image is None to avoid error Image 0 is processed
            resized=cv2.resize(img0 ,(32,32), interpolation= cv2.INTER_AREA)
            return torch.from_numpy(resized.transpose()).float(),torch.rand(1)
        else:
            resized=cv2.resize(img ,(32,32), interpolation= cv2.INTER_AREA)
            return torch.from_numpy(resized.transpose()).float(),torch.rand(1)       

https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

#### Divide these resulting arrays into corresponding train/validation/test splits. Leave the last 10k images for testing (images are id’ed). You are free to define the length of the validation split.

In [5]:
dataset = Dataset()
validation_split = .2
test_split=0.22
shuffle_dataset = True
random_seed= 3116

# Creating data indices for training and test splits:
dataset_size = len(dataset)
# print(dataset_size)
indices = list(range(dataset_size))
# split = int(np.floor(validation_split * dataset_size))
split = int(np.floor(test_split * dataset_size)) #This leads to having 10k images for testing
split_val = int(np.floor(validation_split * dataset_size)) #Validation Split size is 0.2*training_set

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
# train_indices, val_indices = indices[split:], indices[:split]
train_indices, test_indices = indices[split:], indices[:split]
train_indices, val_indices = train_indices[split_val:], train_indices[:split_val]

print("Length of Training Samples is :",len(train_indices))
print("Length of Validation Samples is :",len(val_indices))
print("Length of Testing Samples is :",len(test_indices))
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader1 = torch.utils.data.DataLoader(dataset, batch_size=60,sampler=train_sampler)
validation_loader1 = torch.utils.data.DataLoader(dataset, batch_size=60,sampler=valid_sampler)
test_loader1 = torch.utils.data.DataLoader(dataset, batch_size=60,sampler=test_sampler)

Length of Training Samples is : 26431
Length of Validation Samples is : 9114
Length of Testing Samples is : 10025


In [6]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
        #Here convulutional network is implemented from the paper End to End Learning for Self-Driving Cars
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5 , stride=2),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 3, padding=1),
            nn.ELU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.Dropout(0.5)
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )
        
    def forward(self, input):
        """Forward pass."""
#         print("Input shape is",input.shape)
        input = input.view(input.size(0),3,32,32)
        
        output = self.conv_layers(input)
        # print(output.shape)
        output = output.view(output.size(0), -1)
#         print(output.shape)
        output = self.linear_layers(output)
        return output

In [7]:
net=ConvNet()
optimizer= torch.optim.Adam(net.parameters(), lr=1e-3)
criterion= torch.nn.MSELoss()
train_loss=0.0
for i, sample_batched in enumerate(train_loader1):
    optimizer.zero_grad()
    yhat= net(sample_batched[0])
    loss= criterion(yhat.squeeze(), torch.rand(1))
    loss.backward()
    optimizer.step()
    
    train_loss += loss.data.item()
    if i % 100 == 0:
            print("Training Epoch: {} | Loss: {}".format(1, train_loss / (i + 1)))

    
valid_loss = 0
with torch.no_grad():
    for i, sample_batched in enumerate(validation_loader1):
        optimizer.zero_grad()
        yhat= net(sample_batched[0])
        loss= criterion(yhat.squeeze(), torch.rand(1))

        valid_loss+=loss.data.item()

        if i % 100 == 0:
                print("Validation Loss: {}".format(valid_loss / (i + 1)))
 
test_loss=0
with torch.no_grad():
    for i, sample_batched in enumerate(test_loader1):
        optimizer.zero_grad()
        yhat= net(sample_batched[0])
        loss= criterion(yhat.squeeze(), torch.rand(1))

        test_loss+=loss.data.item()

        if i % 10025 == 0:
                print("Total Test RMSE Loss: {}".format(sqrt(test_loss / (i + 1))))
    

        


Training Epoch: 1 | Loss: 0.40316832065582275


Premature end of JPEG file


Wrong path: /kaggle/input/car-steering-angle-prediction/driving_dataset/
Training Epoch: 1 | Loss: 0.11238230965422964
Training Epoch: 1 | Loss: 0.09977858130624911
Training Epoch: 1 | Loss: 0.09920076517975329
Training Epoch: 1 | Loss: 0.09554425421125311
Validation Loss: 0.07495982199907303
Validation Loss: 0.08954285485479788
Total Test RMSE Loss: 0.12917807846520685
Wrong path: /kaggle/input/car-steering-angle-prediction/driving_dataset/


### Thus we can see Total Test RMSE Loss: 0.12917807846520685 on the last 10k images in Test Set

#### References:
 - https://github.com/Zhenye-Na/e2e-learning-self-driving-cars
 - Exercise Sheet Sample Code
 